In [1]:
import pandas as pd
import numpy as np
import datetime
import os

In [2]:
def get_filenames():
    filenames = []

    for _, _, files in os.walk("./routes"):
        for file in files:
            if file.endswith(".csv"):
                filenames.append(file)
    return filenames

csv_filenames = get_filenames()

In [3]:
def to_time(datetime_string):
    d = datetime.datetime.strptime(datetime_string, "%Y-%m-%d %H:%M:%S.%f")+datetime.timedelta(hours=-5)
    return d.strftime("%H")

def to_mexican(dt_string):
    d = datetime.datetime.strptime(dt_string, "%Y-%m-%d %H:%M:%S.%f")+datetime.timedelta(hours=-5)
    return d

complete_data = None

for i, filename in enumerate(csv_filenames):
    data = pd.read_csv(f"./routes/{filename}")
    data["hour"] = data["time"].apply(to_time)
    data["time"] = data["time"].apply(to_mexican)
    
    if(int(data['hour'][0]) > 21 or int(data['hour'][0]) < 5):
        continue
    if complete_data is None:
        complete_data = data
    else:
        complete_data = pd.merge(complete_data, data, how='outer')

In [4]:
complete_data = complete_data.drop(['diff'], axis=1)

In [5]:
complete_data['traffic_proportion'] = complete_data['duration_with_traffic']/complete_data['duration']

In [6]:
def q1(x):
    return x.quantile(0.25)
def q3(x):
    return x.quantile(0.75)
def std_shift(x):
    return x.mean() + (0.75*x.std())

f = {'traffic_proportion': ['mean', 'median', 'std', q1, q3, std_shift]}

grouped_data = complete_data.groupby('name').agg(f)
grouped_data

traffic_proportion                                \
                                   mean    median       std        q1   
name                                                                    
Alcalde                        1.485660  1.549822  0.262362  1.350534   
Avila_Camacho                  1.826838  1.829167  0.264496  1.745833   
Enrique_Diaz_de_Leon           1.994142  2.078838  0.297886  1.950207   
Federalismo                    2.309606  2.422969  0.339313  2.207283   
Hidalgo                        2.521228  2.836991  0.903050  1.583072   
Independencia                  2.182309  2.258813  0.345468  1.964750   
Juarez                         2.247712  2.397222  0.579471  1.861111   
Lazaro_Cardenas                1.608958  1.567237  0.356551  1.369193   
Lopez_Mateos                   1.960836  1.605634  0.764347  1.453834   
Vallarta                       1.766233  1.809264  0.385587  1.534060   

                                          
                            q3 std_shift  
name                                      
Alcalde               1.647687  1.682432  
Avila_Camacho         2.014583  2.025210  
Enrique_Diaz_de_Leon  2.161826  2.217557  
Federalismo           2.607843  2.564091  
Hidalgo               3.241379  3.198516  
Independencia         2.474954  2.441410  
Juarez                2.550000  2.682315  
Lazaro_Cardenas       1.841076  1.876371  
Lopez_Mateos          2.469484  2.534096  
Vallarta              2.068120  2.055423

In [7]:
complete_data['has_traffic'] = [1 if row['traffic_proportion'] > grouped_data['traffic_proportion'].loc[row['name']]['std_shift'] else 0  for i, row in complete_data.iterrows()] 

In [8]:
complete_data

,name,time,duration,duration_with_traffic,hour,traffic_proportion,has_traffic
0,Vallarta,2019-05-03 17:44:21.350509,6.116667,12.650000,17,2.068120,1
1,Juarez,2019-05-03 17:44:21.350509,6.000000,16.016667,17,2.669444,0
2,Hidalgo,2019-05-03 17:44:21.350509,5.316667,17.800000,17,3.347962,1
3,Federalismo,2019-05-03 17:44:21.350509,5.950000,14.416667,17,2.422969,0
4,Alcalde,2019-05-03 17:44:21.350509,9.366667,13.083333,17,1.396797,0
5,Enrique_Diaz_de_Leon,2019-05-03 17:44:21.350509,8.033333,16.700000,17,2.078838,0
6,Lopez_Mateos,2019-05-03 17:44:21.350509,10.650000,27.016667,17,2.536776,1
7,Avila_Camacho,2019-05-03 17:44:21.350509,8.000000,14.683333,17,1.835417,0
8,Independencia,2019-05-03 17:44:21.350509,17.966667,39.000000,17,2.170686,0
9,Lazaro_Cardenas,2019-05-03 17:44:21.350509,6.816667,12.750000,17,1.870416,0


In [9]:
complete_data.groupby('name')['has_traffic'].sum()

name
Alcalde                 4
Avila_Camacho           4
Enrique_Diaz_de_Leon    1
Federalismo             5
Hidalgo                 5
Independencia           5
Juarez                  3
Lazaro_Cardenas         3
Lopez_Mateos            4
Vallarta                5
Name: has_traffic, dtype: int64

In [10]:
complete_data.head(10)

,name,time,duration,duration_with_traffic,hour,traffic_proportion,has_traffic
0,Vallarta,2019-05-03 17:44:21.350509,6.116667,12.650000,17,2.068120,1
1,Juarez,2019-05-03 17:44:21.350509,6.000000,16.016667,17,2.669444,0
2,Hidalgo,2019-05-03 17:44:21.350509,5.316667,17.800000,17,3.347962,1
3,Federalismo,2019-05-03 17:44:21.350509,5.950000,14.416667,17,2.422969,0
4,Alcalde,2019-05-03 17:44:21.350509,9.366667,13.083333,17,1.396797,0
5,Enrique_Diaz_de_Leon,2019-05-03 17:44:21.350509,8.033333,16.700000,17,2.078838,0
6,Lopez_Mateos,2019-05-03 17:44:21.350509,10.650000,27.016667,17,2.536776,1
7,Avila_Camacho,2019-05-03 17:44:21.350509,8.000000,14.683333,17,1.835417,0
8,Independencia,2019-05-03 17:44:21.350509,17.966667,39.000000,17,2.170686,0
9,Lazaro_Cardenas,2019-05-03 17:44:21.350509,6.816667,12.750000,17,1.870416,0


In [11]:
indexed = complete_data.set_index('time')

In [12]:
indexed = indexed.drop(['duration', 'duration_with_traffic', 'traffic_proportion'], axis=1)

In [13]:
indexed = pd.pivot_table(indexed, index=["time", "hour"], columns='name')['has_traffic']

In [14]:
indexed.to_csv('indexed.csv')

In [23]:
indexed.reset_index(inplace=True)
indexed

name,index,time,hour,Alcalde,Avila_Camacho,Enrique_Diaz_de_Leon,Federalismo,Hidalgo,Independencia,Juarez,Lazaro_Cardenas,Lopez_Mateos,Vallarta
0,0,2019-05-03 05:44:21.352348,05,0,0,0,0,0,0,0,0,0,0
1,1,2019-05-03 06:44:21.352269,06,0,0,0,0,0,0,0,0,0,0
2,2,2019-05-03 07:44:21.352264,07,0,0,0,0,0,0,0,0,0,0
3,3,2019-05-03 08:44:21.352268,08,0,0,1,1,0,0,1,0,0,1
4,4,2019-05-03 09:44:21.352290,09,0,0,0,0,0,0,0,0,0,0
5,5,2019-05-03 10:44:21.352273,10,0,0,0,0,0,0,0,0,0,0
6,6,2019-05-03 11:44:21.350552,11,1,0,0,0,0,0,0,0,0,0
7,7,2019-05-03 12:44:21.350518,12,1,0,0,0,1,1,0,0,0,0
8,8,2019-05-03 13:44:21.350544,13,1,1,0,1,1,1,0,0,0,1
9,9,2019-05-03 14:44:21.350528,14,1,1,0,1,0,1,0,1,1,0


In [16]:
checkpoints = [
    "Hospicio Cabañas",
    "Plaza Galerías",
    "Andares",
    "Secretaría de Movilidad",
    "La Minerva",
    "Basílica de Zapopan"
]

In [17]:
i_rows = []
for i, r in indexed.iterrows():
    for origin in checkpoints:
        destinations = [i for i in checkpoints if i != origin]
        for destination in destinations:
            i_rows.append(r.append(pd.Series({"origin": origin, "destination": destination})))
i_rows[0]

time                    2019-05-03 05:44:21.352348
hour                                            05
Alcalde                                          0
Avila_Camacho                                    0
Enrique_Diaz_de_Leon                             0
Federalismo                                      0
Hidalgo                                          0
Independencia                                    0
Juarez                                           0
Lazaro_Cardenas                                  0
Lopez_Mateos                                     0
Vallarta                                         0
origin                            Hospicio Cabañas
destination                         Plaza Galerías
dtype: object

In [18]:
total_play = pd.DataFrame(i_rows)

In [19]:
total_play

,time,hour,Alcalde,Avila_Camacho,Enrique_Diaz_de_Leon,Federalismo,Hidalgo,Independencia,Juarez,Lazaro_Cardenas,Lopez_Mateos,Vallarta,origin,destination
0,2019-05-03 05:44:21.352348,05,0,0,0,0,0,0,0,0,0,0,Hospicio Cabañas,Plaza Galerías
1,2019-05-03 05:44:21.352348,05,0,0,0,0,0,0,0,0,0,0,Hospicio Cabañas,Andares
2,2019-05-03 05:44:21.352348,05,0,0,0,0,0,0,0,0,0,0,Hospicio Cabañas,Secretaría de Movilidad
3,2019-05-03 05:44:21.352348,05,0,0,0,0,0,0,0,0,0,0,Hospicio Cabañas,La Minerva
4,2019-05-03 05:44:21.352348,05,0,0,0,0,0,0,0,0,0,0,Hospicio Cabañas,Basílica de Zapopan
5,2019-05-03 05:44:21.352348,05,0,0,0,0,0,0,0,0,0,0,Plaza Galerías,Hospicio Cabañas
6,2019-05-03 05:44:21.352348,05,0,0,0,0,0,0,0,0,0,0,Plaza Galerías,Andares
7,2019-05-03 05:44:21.352348,05,0,0,0,0,0,0,0,0,0,0,Plaza Galerías,Secretaría de Movilidad
8,2019-05-03 05:44:21.352348,05,0,0,0,0,0,0,0,0,0,0,Plaza Galerías,La Minerva
9,2019-05-03 05:44:21.352348,05,0,0,0,0,0,0,0,0,0,0,Plaza Galerías,Basílica de Zapopan


In [21]:
total_play.to_csv('total_play.csv')